In [89]:
#import externally developed packages
import os
import sys
import vcfpy
import re
import pandas as pd

## Import a .vcf file and convert it into a dataframe with a unique identifier for each SNP

In [122]:
##import vcf

#specify path of vcf
vcf_path = '/Users/tristanpwdennis/Software/seqtoolbox/analysis-scripts/vcf/vcf/test.vcf'

#import vcf as reader object
reader = vcfpy.Reader.from_path(vcf_path)

# Build header
header = pd.DataFrame.transpose(pd.DataFrame(['#CHROM', 'POS', 'REF', 'ALT'] + reader.header.samples.names))

# Design matrix in list format
X_list = []  

#convert vcf convert to dataframe - extract GT field, chrom, pos ref and alt data
for record in reader:
    if not record.is_snv():
        continue
    line = [record.CHROM, record.POS, record.REF]
    line += [alt.value for alt in record.ALT]
    line += [call.data.get('GT') or './.' for call in record.calls]
    X_list.append(line)
    df = pd.DataFrame(X_list)

#stick header onto the genotype frame
df=header.append(df) 

#make added row into the df header
new_header = df.iloc[0] #grab the first row for the header
df = df[1:] #take the data less the header row
df.columns = new_header #set the header row as the df header

In [123]:
id=df["#CHROM"].map(str) + df["POS"].map(str) + df["REF"].map(str) + df["ALT"].map(str)
df.insert(0, 'id', id)
gt=df.drop(['#CHROM', 'POS', 'REF', 'ALT'], axis=1)

In [129]:
##vcf to colony input

#specify mapping (from genotype values to codes)
di = {
    "./.": 0,
    "0/0": 1,
    "0/1": 2,
    "1/0": 3,
    "1/1": 4,
}  

#make genotype id
gt=gt.replace(di)


In [131]:
gt=gt.transpose()